In [3]:
import pandas as pd
import boto3
import sagemaker

In [2]:
# 1. Create Model Package Group
sm_client = boto3.client('sagemaker')

try:
    response = sm_client.create_model_package_group(
        ModelPackageGroupName='flight-delay-models',
        ModelPackageGroupDescription='Flight delay prediction models for airline operations'
    )
    print(f"✅ Model Package Group created: flight-delay-models")
except sm_client.exceptions.ResourceInUse:
    print("ℹ️  Model Package Group already exists: flight-delay-models")

# 2. Verify it exists
groups = sm_client.list_model_package_groups()
for group in groups['ModelPackageGroupSummaryList']:
    if group['ModelPackageGroupName'] == 'flight-delay-models':
        print(f"✅ Confirmed: {group['ModelPackageGroupName']} exists")
        print(f"   ARN: {group['ModelPackageGroupArn']}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 sm_client = boto3.client('sagemaker')                                                       │
│    6                                                                                             │
│    7 try:                                                                                        │
│ ❱  8 │   response = sm_client.create_model_package_group(                                        │
│    9 │   │   ModelPackageGroupName='flight-delay-models',                                        │
│   10 │   │   ModelPackageGroupDescription='Flight delay prediction models for airline operati    │
│   11 │   )                                                                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ClientError: An error occurred (ValidationException) when calling the CreateModelPackageGroup operation: Model 
Package Group already exists: arn:aws:sagemaker:us-east-1:425709451100:model-package-group/flight-delay-models

In [4]:
# ===========================
# CONFIGURATION
# ===========================
bucket = 'sagemaker-us-east-1-425709451100'
prefix = 'aai540-group1'
target_col = 'DELAYED' 

# S3 paths
train_parquet = f's3://{bucket}/{prefix}/features/train_features.parquet'
val_parquet = f's3://{bucket}/{prefix}/features/val_features.parquet'
test_parquet = f's3://{bucket}/{prefix}/features/test_features.parquet'

# ===========================
# LOAD DATA
# ===========================
print("Loading Parquet files from S3...")
train_df = pd.read_parquet(train_parquet)
val_df = pd.read_parquet(val_parquet)
test_df = pd.read_parquet(test_parquet)

print(f"✅ Train: {train_df.shape[0]:,} rows × {train_df.shape[1]} columns")
print(f"✅ Val:   {val_df.shape[0]:,} rows × {val_df.shape[1]} columns")
print(f"✅ Test:  {test_df.shape[0]:,} rows × {test_df.shape[1]} columns")

print(f"\n📋 Columns: {train_df.columns.tolist()}")

# ===========================
# REORDER COLUMNS (TARGET FIRST)
# ===========================
print(f"\n🎯 Target column: '{target_col}'")

# Get feature columns (everything except target)
feature_cols = [col for col in train_df.columns if col != target_col]

# Reorder: target first, then features
train_df = train_df[[target_col] + feature_cols]
val_df = val_df[[target_col] + feature_cols]
test_df = test_df[[target_col] + feature_cols]

print(f"✅ Reordered: First column is now '{train_df.columns[0]}'")
print(f"   Features: {train_df.columns[1:].tolist()}")

# ===========================
# SAVE AS CSV (NO HEADER, NO INDEX)
# ===========================
print("\n💾 Saving as CSV...")
train_df.to_csv('train.csv', header=False, index=False)
val_df.to_csv('val.csv', header=False, index=False)
test_df.to_csv('test.csv', header=False, index=False)

print("✅ CSV files created locally")

# Show sample
print("\n📄 First row of train.csv (first 10 values):")
print(f"   {train_df.iloc[0, :10].values}")

# ===========================
# UPLOAD TO S3
# ===========================
print("\n☁️  Uploading to S3...")
s3 = boto3.client('s3')

s3.upload_file('train.csv', bucket, f'{prefix}/data/train/train.csv')
s3.upload_file('val.csv', bucket, f'{prefix}/data/validation/val.csv')
s3.upload_file('test.csv', bucket, f'{prefix}/data/test/test.csv')

print("✅ Files uploaded:")
print(f"   📁 s3://{bucket}/{prefix}/data/train/train.csv")
print(f"   📁 s3://{bucket}/{prefix}/data/validation/val.csv")
print(f"   📁 s3://{bucket}/{prefix}/data/test/test.csv")

# ===========================
# VERIFY UPLOADS
# ===========================
print("\n🔍 Verifying S3 uploads...")
for folder in ['train', 'validation', 'test']:
    response = s3.list_objects_v2(
        Bucket=bucket, 
        Prefix=f'{prefix}/data/{folder}/'
    )
    for obj in response.get('Contents', []):
        size_mb = obj['Size'] / (1024 * 1024)
        print(f"   ✅ {folder}: {size_mb:.1f} MB")

print("\n" + "="*70)
print("🎉 COMPLETE - DATA READY FOR SAGEMAKER TRAINING")
print("="*70)
print(f"   Training:   s3://{bucket}/{prefix}/data/train/")
print(f"   Validation: s3://{bucket}/{prefix}/data/validation/")
print(f"   Test:       s3://{bucket}/{prefix}/data/test/")
print(f"\n🎯 Target: {target_col}")
print(f"📊 Features: {len(feature_cols)} features")

Loading Parquet files from S3...
✅ Train: 4,299,046 rows × 27 columns
✅ Val:   482,878 rows × 27 columns
✅ Test:  462,367 rows × 27 columns

📋 Columns: ['MONTH', 'DAY', 'DAY_OF_WEEK', 'SCHEDULED_DEPARTURE', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DISTANCE', 'SCHEDULED_TIME', 'DELAYED', 'DEP_HOUR', 'HOUR_SIN', 'HOUR_COS', 'IS_PEAK_HOUR', 'IS_WEEKEND', 'IS_LONG_HAUL', 'DISTANCE_BUCKET', 'ROUTE', 'AIRLINE_DELAY_RATE', 'ORIGIN_DELAY_RATE', 'DEST_DELAY_RATE', 'ROUTE_DELAY_RATE', 'ORIGIN_FLIGHTS', 'DEST_FLIGHTS', 'ROUTE_FLIGHTS', 'flight_id', 'event_time']

🎯 Target column: 'DELAYED'
✅ Reordered: First column is now 'DELAYED'
   Features: ['MONTH', 'DAY', 'DAY_OF_WEEK', 'SCHEDULED_DEPARTURE', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DISTANCE', 'SCHEDULED_TIME', 'DEP_HOUR', 'HOUR_SIN', 'HOUR_COS', 'IS_PEAK_HOUR', 'IS_WEEKEND', 'IS_LONG_HAUL', 'DISTANCE_BUCKET', 'ROUTE', 'AIRLINE_DELAY_RATE', 'ORIGIN_DELAY_RATE', 'DEST_DELAY_RATE', 'ROUTE_DELAY_RATE', 'ORIGIN_FLIGHTS',